In [1]:
import numpy as np  # for handling multi-dimensional array operation
import pandas as pd  # for reading data from csv 
# import statsmodels.api as sm  # for finding the p-value~
from sklearn.preprocessing import MinMaxScaler  # for normalization
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score 
from sklearn.metrics import recall_score 
from sklearn.utils import shuffle
# >> FEATURE SELECTION << #
#def remove_correlated_features(X):
#def remove_less_significant_features(X, Y):
reg_strength = 10000 # regularization strength
learning_rate = 0.000001

In [2]:
def compute_cost(W, X, Y):
    # calculate hinge loss
    N = X.shape[0]
    distances = 1 - Y * (np.dot(X, W))
    distances[distances < 0] = 0  # equivalent to max(0, distance)
    hinge_loss = reg_strength * (np.sum(distances) / N)
    
    # calculate cost
    cost = 1 / 2 * np.dot(W, W) + hinge_loss
    return cost

In [6]:
def calculate_cost_gradient(W, X_batch, Y_batch):
    # if only one example is passed (eg. in case of SGD)
    Y_batch = np.array([Y_batch])
    X_batch = np.array([X_batch])
    distance = 1 - (Y_batch * np.dot(X_batch, W))
    dw = np.zeros(len(W))    
    for ind, d in enumerate(distance):
        if max(0, d) == 0:
            di = W
        else:
            di = W - (reg_strength * Y_batch[ind] * X_batch[ind])
        dw += di    
    dw = dw/len(Y_batch)  # average
    return dw

In [4]:
def sgd(features, outputs):
    max_epochs = 5000
    weights = np.zeros(features.shape[1])
    nth = 0
    prev_cost = float("inf")
    cost_threshold = 0.01  # in percent
    # stochastic gradient descent
    for epoch in range(1, max_epochs): 
        # shuffle to prevent repeating update cycles
        X, Y = shuffle(features, outputs)
        for ind, x in enumerate(X):
            ascent = calculate_cost_gradient(weights, x, Y[ind])
            weights = weights - (learning_rate * ascent)
        
        # convergence check on 2^nth epoch
        if epoch == 2 ** nth or epoch == max_epochs - 1:
            cost = compute_cost(weights, features, outputs)
            print("Epoch is:{} and Cost is: {}".format(epoch, cost))
            # stoppage criterion
            if abs(prev_cost - cost) < cost_threshold * prev_cost:
                return weights
            prev_cost = cost
            nth += 1
    return weights

In [7]:

data = pd.read_csv('data.csv')    # SVM only accepts numerical values. 
# Therefore, we will transform the categories M and B into
# values 1 and -1 (or -1 and 1), respectively.
diagnosis_map = {'M':1, 'B':-1}
data['diagnosis'] = data['diagnosis'].map(diagnosis_map)    # drop last column (extra column added by pd)
# and unnecessary first column (id)
data.drop(data.columns[[-1, 0]], axis=1, inplace=True)
# put features & outputs in different DataFrames for convenience
Y = data.loc[:, 'diagnosis']  # all rows of 'diagnosis' 
X = data.iloc[:, 1:]  # all rows of column 1 and ahead (features)# normalize the features using MinMaxScalar from
# sklearn.preprocessing
X_normalized = MinMaxScaler().fit_transform(X.values)
X = pd.DataFrame(X_normalized)
# first insert 1 in every row for intercept b
X.insert(loc=len(X.columns), column='intercept', value=1)# test_size is the portion of data that will go into test set
# random_state is the seed used by the random number generator
print("splitting dataset into train and test sets...")
X_train, X_test, y_train, y_test = tts(X, Y, test_size=0.2, random_state=42)
print("training started...")
W = sgd(X_train.to_numpy(), y_train.to_numpy())
print("training finished.")
print("weights are: {}".format(W))
# testing the model on test set
y_test_predicted = np.array([])
for i in range(X_test.shape[0]):
    yp = np.sign(np.dot(W, X_test.to_numpy()[i])) #model
    y_test_predicted = np.append(y_test_predicted, yp)
print("accuracy on test dataset: {}".format(accuracy_score(y_test.to_numpy(), y_test_predicted)))
print("recall on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))
print("precision on test dataset: {}".format(recall_score(y_test.to_numpy(), y_test_predicted)))

splitting dataset into train and test sets...
training started...
Epoch is:1 and Cost is: 5283.266288216451
Epoch is:2 and Cost is: 3494.296312958688
Epoch is:4 and Cost is: 2495.0194801604516
Epoch is:8 and Cost is: 1995.6858160981185
Epoch is:16 and Cost is: 1519.139170197456
Epoch is:32 and Cost is: 1205.9199867880086
Epoch is:64 and Cost is: 978.5145459587268
Epoch is:128 and Cost is: 808.4396463563917
Epoch is:256 and Cost is: 702.2289718043986
Epoch is:512 and Cost is: 645.023663866896
Epoch is:1024 and Cost is: 622.5571292392117
Epoch is:2048 and Cost is: 615.7340160514601
Epoch is:4096 and Cost is: 608.8037498570799
Epoch is:4999 and Cost is: 610.2595160585483
training finished.
weights are: [ 1.34545572  0.84283661  1.14446453  2.17208192 -1.23776451 -3.22334476
  3.26791681  6.81875633 -0.46270825  0.1216231   5.67888346 -1.92231947
  3.27334311  3.76862072  1.66252483 -2.42774398 -1.757855    0.85248739
 -1.96579726 -1.85864575  2.71350387  5.3236799   1.05697928  3.0852694


In [ ]:
a=10
type(a)

int